### This notebook trains a yolo detection model to recognize beetles and scale bars in images

In [4]:
# import pip
# pip.main(["install", "ultralytics"])

In [13]:
import os, glob, shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ultralytics import YOLO
import torch
import cv2
import wget

In [9]:
#access GPU if available
use_cuda = torch.cuda.is_available()
DEVICE   = torch.device("cuda:0" if use_cuda else "cpu")
print("Device: ", DEVICE)

if use_cuda:
    print('__CUDNN VERSION:', torch.backends.cudnn.version())
    print('__Number CUDA Devices:', torch.cuda.device_count())
    print('__CUDA Device Name:',torch.cuda.get_device_name(0))
    print('__CUDA Device Total Memory [GB]:',torch.cuda.get_device_properties(0).total_memory/1e9)

Device:  cuda:0
__CUDNN VERSION: 90100
__Number CUDA Devices: 8
__CUDA Device Name: NVIDIA RTX A6000
__CUDA Device Total Memory [GB]: 51.033931776


In [10]:
#helper functions
def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0,0,0,0), lw=2))

In [25]:
#define path to yaml file containing info about our dataset
YAML     = '/home/ramirez.528/2018-NEON-beetles-processing/detection/data/beetle_detection.yaml'
EPOCHS   = 25

In [15]:
#download the pretrained yolo_v8m weights
yolo_8m = wget.download('https://github.com/ultralytics/assets/releases/download/v8.2.0/yolov8m.pt')
yolo_8m

'yolov8m.pt'

In [27]:
# Load a pretrained model
model = YOLO(yolo_8m)

# Train the model
outputs_folder = '/home/ramirez.528/2018-NEON-beetles-processing/detection/runs'
save_name = 'beetles_29_train_images_yolo_8m_shear_10.0_scale_0.5_translate_0.1_fliplr_0.2_flipud_0.2' ##USE THIS ONE
results = model.train(data=YAML, 
                      epochs=EPOCHS, 
                      batch=4,
                      device=DEVICE,
                      val=True,
                      optimizer='auto',
                      verbose=True,
                      project=outputs_folder,
                      name = save_name,
                      shear=10.0,
                      scale=0.5, 
                      translate=0.1,
                      fliplr = 0.2,
                      flipud = 0.2
                     )



New https://pypi.org/project/ultralytics/8.2.77 available 😃 Update with 'pip install -U ultralytics'
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/home/ramirez.528/2018-NEON-beetles-processing/detection/data/beetle_detection.yaml, epochs=25, time=None, patience=100, batch=4, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=/home/ramirez.528/2018-NEON-beetles-processing/detection/runs, name=beetles_29_train_images_yolo_8m_shear_10.0_scale_0.5_translate_0.1_fliplr_0.2_flipud_0.28, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visua

100%|██████████| 755k/755k [00:00<00:00, 16.4MB/s]

Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics

  9                  -1  1    831168  ultralytics.nn.modules.block.SPPF            [576, 576, 5]                 
 10                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 11             [-1, 6]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 12                  -1  2   1993728  ultralytics.nn.modules.block.C2f             [960, 384, 2]                 
 13                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 14             [-1, 4]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 15                  -1  2    517632  ultralytics.nn.modules.block.C2f             [576, 192, 2]                 
 16                  -1  1    332160  ultralytics.nn.modules.conv.Conv             [192, 192, 3, 2]              
 17            [-1, 12]  1         0  ultralytics.nn.modules.conv.Concat           [1]  

100%|██████████| 6.25M/6.25M [00:00<00:00, 81.5MB/s]


...

train: New cache created: /home/ramirez.528/2018-NEON-beetles-processing/detection/data/labels/train.cache



val: Scanning /home/ramirez.528/2018-NEON-beetles-processing/detection/data/labels/train.cache... 29 images, 0 backgrounds, 0 corrupt: 100%|██████████| 29/29 [00:00<?, ?it/s]


Plotting labels to /home/ramirez.528/2018-NEON-beetles-processing/detection/runs/beetles_29_train_images_yolo_8m_shear_10.0_scale_0.5_translate_0.1_fliplr_0.2_flipud_0.28/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 


/home/ramirez.528/anaconda3/envs/beetles/lib/python3.10/site-packages/_distutils_hack/__init__.py:11: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/home/ramirez.528/anaconda3/envs/beetles/lib/python3.10/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


AssertionError: /home/ramirez.528/anaconda3/envs/beetles/lib/python3.10/distutils/core.py